In [1]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

c:\users\ramon\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
def remove_prefix_from_name(name):

    name = name + ' '
    
    suffixes = [' BBA ', ' LLB ', " MSc ", " MA ", " BA ", " RA "]
    for suffix in suffixes:
        if suffix in name:
            name = name.replace(suffix[:-1], suffix[:-1]+'.')

    
    
    pattern = r"(?!ij|th|ph|hr|ch|jr|sr)[a-z]{2,}\.\s?"
    prefixes = re.findall(pattern, name, flags=re.IGNORECASE)
    
    if prefixes:
        name_without_prefixes = re.sub(pattern, '', name, flags=re.IGNORECASE).strip()
        
        
        return prefixes, name_without_prefixes
    else:
        return '', name.strip()
    
print(remove_prefix_from_name('Dr. J. Daane MSc'))

(['Dr. ', 'MSc. '], 'J. Daane')


In [3]:
def xpath(element, path, namespaces):
    obj = element.find(path, namespaces)
    
    if obj is not None:
        return obj.text
    else:
        return ''

In [4]:
def xpath_adres(element, namespaces, Type):
    
    Incomplete = False
    
    if Type == 'Bezoek':
    
        obj = element.find('p:adressen/p:adres[p:type="Bezoekadres"]', namespaces)

        if obj is not None:
            
            straat = obj.find('p:straat', namespaces)
            if straat is not None:
                straat = straat.text
            else:
                Incomplete = True
            
            huisnummer = obj.find('p:huisnummer', namespaces)
            if huisnummer is not None:
                huisnummer = huisnummer.text
            else:
                Incomplete = True
            
            postcode = obj.find('p:postcode', namespaces)
            if postcode is not None:
                postcode = postcode.text
            else:
                Incomplete = True
                
            plaats = obj.find('p:plaats', namespaces)
            if plaats is not None:
                plaats = plaats.text.title()
                
            else:
                Incomplete = True
                
            if Incomplete == True:
                return ''

            return f"{straat} {huisnummer} {postcode} {plaats}"
         
        else:
            return ''
        
    if Type == 'Post':
        
        obj = element.find('p:adressen/p:adres[p:type="Postadres"]', namespaces)
        
        if obj is not None:
            postbus = obj.find('p:postbus', namespaces)
            if postbus is not None:
                postbus = postbus.text
            else:
                Incomplete = True
            
            postcode = obj.find('p:postcode', namespaces)
            if postcode is not None:
                postcode = postcode.text
            else:
                Incomplete = True
            
            plaats = obj.find('p:plaats', namespaces)
            if plaats is not None:
                plaats = plaats.text.title()
                
            else:
                Incomplete = True
                
            if Incomplete == True:
                return ''
            

            return f"{postbus} {postcode} {plaats}"
         
        else:
            return ''
        
    else:
        return ''
        
        

In [5]:
def xpath_TOOI(element, path, namespaces, afkorting):
    
    dc_publisher = element.find(path, namespaces)
    if dc_publisher is not None:

        # hier wordt de afkorting gebruikt bijv. ws = waterschappen
        if dc_publisher.text.rsplit('/')[-1][:len(afkorting)] == afkorting:
            
            return dc_publisher.text.rsplit('/')[-1]
        else:
            return ''
    else:
        return ''
    

Uitdagingen namen: prefixes als Ch. Chr. Th. Ph. zijn eigenlijk initialen. Wat te doen met initialen zonder puntjes. 

In [6]:
def xpath_naam(element, path, namespaces):
    # verzamel de naam in de XML
    initial_name = element.find(path, namespaces)
    
    if initial_name is not None:
        initial_name_txt = initial_name.text
        
        pattern = r'\((.*?)\)'
        matches = re.findall(pattern, initial_name_txt)
        
        initial_name_txt = re.sub(pattern, '', initial_name_txt)
        if " (" in initial_name_txt:
            initial_name_txt = initial_name_txt.split(' (')[0]
        
        if '  ' in initial_name_txt:
            initial_name_txt = initial_name_txt.replace('  ', ' ')

        # er staat vaak een prefix als dr. of mew. voor deze wordt verwijderd?
        prefixes, name_without_prefix = remove_prefix_from_name(initial_name_txt)
        name_without_prefix = name_without_prefix.lstrip()

        initial_name_split = name_without_prefix.split(' ', 1)

        # bij gemeenten staat er een persoon in als mew. Schouten dus zonder initialen. 
        if len(initial_name_split) == 2:
            first_word = initial_name_split[0]
            if '.' not in first_word and len(first_word) != 1:

                foaf_firstName = initial_name_split[0]
                foaf_initials = foaf_firstName[0].upper() + '.'
            else:
                foaf_initials = initial_name_split[0]
                foaf_firstName = ''

            foaf_lastName = initial_name_split[1]
            x = foaf_lastName.split(' ')
            if x[0] != '':
                if x[0][-1] == '.':
                    foaf_lastName = ''.join(x[1:])
                    foaf_initials = foaf_initials + x[0]
        
        else:
            foaf_initials = ''
            foaf_firstName = ''
            foaf_lastName = initial_name_split[0]
    
    # gebeurt in de praktijk niet maar maakt het wel failproof
    else:
        foaf_initials = ''
        foaf_firstName = ''
        foaf_lastName = ''
        
    prefixes = ''.join(prefixes)
    prefixes = prefixes.rstrip()
    
    if matches != []:
        if len(matches[0].split(' ')) == 1:
            foaf_firstName = matches[0]
            
            if '.' in foaf_firstName:
                foaf_firstName = ''
        
    if foaf_firstName != '':
        name_without_prefix = foaf_firstName + ' ' + foaf_lastName
        
    if foaf_firstName in ['van', 'De']:
        
        foaf_lastName = foaf_firstName + ' ' + foaf_lastName
        foaf_firstName = ''
        
    if foaf_firstName.isupper() == True:
        foaf_initials = '.'.join(foaf_firstName)
        foaf_firstName = ''
       
    return foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes

In [7]:
def create_organisatie_json(organisatie, namespaces, afkorting):
    
    foi_bezoekadres = xpath_adres(organisatie, namespaces, 'Bezoek')
    foi_postadres = xpath_adres(organisatie, namespaces, 'Post')
    
    foaf_phone = xpath(organisatie, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

    foaf_mbox = xpath(organisatie, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
    
    foi_fax = xpath(organisatie, 'p:contact/p:fax', namespaces)
    
    foi_lastUpdate = xpath(organisatie, 'p:datumMutatie', namespaces)
    
    dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
    dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

    website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

    Type = xpath(organisatie, 'p:types/p:type', namespaces)
    
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    dc_date_year = foi_retrievedDate[:4]
    
    foi_startDate = xpath(organisatie, 'p:startDatum', namespaces)
    
    foi_endDate = xpath(organisatie, 'p:eindDatum', namespaces)
    
    Dict = {
        'dc_identifier': f"nl.{dc_publisher}",
        'dc_title': f"{Type} - {dc_publisher_name}",
        'dc_publisher': dc_publisher,
        'dc_creator': "Ramon Duursma",
        'foi_retrievedDate': foi_retrievedDate,
        'dc_date_year': dc_date_year,
        'dc_publisher_name': dc_publisher_name,
        'foi_website': website_txt,
        'foaf_mbox': foaf_mbox,
        'foaf_phone': foaf_phone,
        'foi_bezoekadres': foi_bezoekadres,
        'foi_postadres': foi_postadres,
        'foi_fax': foi_fax,
        'foi_startDate': foi_startDate,
        'foi_endDate': foi_endDate,
        'foi_lastUpdate': foi_lastUpdate,
        'foaf_type': Type,
        'foi_files': [],
    }
    
    
    filtered_dict = {key: value for key, value in Dict.items() if value != ""}
    return filtered_dict
    

In [8]:
import string

In [9]:
def json_create_single_layer(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',
                   'Agentschap': 'oorg',
            
                  }
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]


    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)
    
    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        org_json = create_organisatie_json(organisatie, namespaces, afkorting)
        
        # in het functie_element staan alle verschillende functies
        functie_element = organisatie.findall('p:functies/p:functie', namespaces)

        # loop over deze functies
        for functie in functie_element:

            foi_count = 0
            DICT = {}

            foaf_function_type = xpath(functie, 'p:naam', namespaces)

            # vindt alle medewerkers die bij deze functie horen
            medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

            # loop over alle medewerkers om per medewerker gegevens te verzamelen
            for medewerker in medewerker_element:
                
                foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)
                
                # optioneel startdatum, telefoonnummer en mailadres
                foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
                
                foi_bezoekadres = xpath_adres(medewerker, namespaces, 'Bezoek')
                
                foi_postadres = xpath_adres(medewerker, namespaces, 'Post')
                
                # genereer beschikbaarheidsgegevens
                if organisatie_type == 'Waterschap':
                    bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {Type.lower()} {dc_publisher_name}"
                else:
                    if foi_party != '':
                        bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                    else:
                        bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"
                
#                 query = f"{name_without_prefix} {foaf_function_type} '{dc_publisher_name}'"
                
#                 hrefs = search_yahoo(query)
    
#                 foi_twitter, foi_website, foi_linkedin = extract_urls(hrefs, website_txt, foaf_lastName)
        
#                 if foi_linkedin != '':
#                     if '/in/' not in foi_linkedin:
#                         foi_linkedin = ''
                    
#                     if '/' in foi_linkedin:
#                         if foi_linkedin.split('/')[-1][0] != foaf_initials[0].lower():
#                             foi_linkedin = ''
                
#                 if foi_website != '':
#                     last_url = foi_website.split('/')
#                     last_url = [item for item in last_url if item]
        
#                     words_on_last_url = last_url[-1].split('-')
                    
#                     score = 0
#                     items = [foaf_lastName, foaf_function_type, dc_publisher_name]
#                     new_list = [split_item.lower() for item in items for split_item in re.split(r'\s|-', item) if split_item]        
#                     new_list = [word.strip(string.punctuation) for word in new_list]
                    
#                     for word in words_on_last_url:
#                         if word not in new_list:
#                             score = score + 1

#                     if score > 2:
#                         foi_website = ''
                                
                    
                # vul de dictionary in
                Dict = {
                        'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                        'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                        'dc_type': Type,
                        'dc_description': bereikbaarheidsgegevens,
                        'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{organisatie_type.lower()}.xml",
                        'dc_publisher': dc_publisher,
                        'dc_creator': "R0m4ndu",
                        'foi_retrievedDate': foi_retrievedDate,
                        'dc_date_year': dc_date_year,
#                         'foi_worksFor': f"nl.{dc_publisher}"
                        'dc_publisher_name': dc_publisher_name,
                        'foi_title': prefixes,
                        'foaf_initials': foaf_initials,
                        'foaf_firstName': foaf_firstName,
                        'foaf_lastName': foaf_lastName,
                        'foaf_name': name_without_prefix,
                        'foaf_mbox': foaf_mbox,
                        'foaf_phone': foaf_phone,
                        'foi_visitAddress': foi_bezoekadres,
                        'foi_mailAddress': foi_postadres,
#                         'foi_linkedin': foi_linkedin,
#                         'foaf_workplaceHomepage': foi_website,
                        'foi_startDate': foi_startDate,
                        'foi_party': foi_party,
                        'foi_function': foaf_function_type,
                        'foi_files': [] ,
                    }
                # Verwijder alle lege strings uit de dict
                filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                # en voeg de dictionary toe aan 
                DICT[foi_count] = filtered_dict

                foi_count+=1
            
            if dc_publisher != '':
            
                # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                            'infobox': {'foi_totalDossiers': len(DICT),
                                         'foi_dossiers': DICT}}


                list_of_dicts.append(final_dict)
                
            

    number_of_people = sum(i['infobox']['foi_totalDossiers'] for i in list_of_dicts)
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}.json", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts


In [10]:
organisatie = 'Gemeente'
people_num, dict_list = json_create_single_layer(organisatie)

tree = ET.parse(f"exportOO_full.xml")
namespaces = {
    'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)
list_of_peoples = [organisatie_element.findall('.//p:medewerker', namespaces) for organisatie_element in organisatie_elements]
people = sum(len(lst) for lst in list_of_peoples)

print(f'aantal medewerkers in XML: {people}')
print(f'aantal medewerkers extracted in JSON: {people_num}')


identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foi_bezoekadres', 'foi_postadres' , 'foi_twitter', 'foaf_workplaceHomepage', 'foi_linkedin', 'dc_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

for dl in dict_list:
    dossiers = dl['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1

print(id_dict)

C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\2287353622.py:28: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Gemeente"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


aantal medewerkers in XML: 14120
aantal medewerkers extracted in JSON: 14120
{'foi_function': 14120, 'dc_publisher': 14120, 'foaf_initials': 14119, 'foaf_firstName': 0, 'foaf_lastName': 14120, 'foi_title': 13998, 'foaf_mbox': 13602, 'foaf_phone': 13758, 'foi_bezoekadres': 0, 'foi_postadres': 0, 'foi_twitter': 0, 'foaf_workplaceHomepage': 0, 'foi_linkedin': 0, 'dc_type': 14120, 'foi_startDate': 0, 'foi_party': 13486}


C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\4012270617.py:9: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Gemeente"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)


In [11]:
def json_create_multi_layer(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Provincie': 'pv',
                  'Ministerie': 'mnre',
                  'Hoog College van Staat': 'oorg',
                  'Rechtspraak': ''}
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse(f"exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)

    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url''p:types/p:type', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        organisatie_elements2 = organisatie.findall('.//p:organisaties/p:organisatie', namespaces)
        
        for organisatie2 in organisatie_elements2:
            
            organisatie_naam = xpath(organisatie2, 'p:naam', namespaces)
        
            # in het functie_element staan alle verschillende functies
            functie_element = organisatie2.findall('p:functies/p:functie', namespaces)


            # loop over deze functies
            for functie in functie_element:
                foi_count = 0
                DICT = {}

                foaf_function_type = xpath(functie, 'p:naam', namespaces)

                # vindt alle medewerkers die bij deze functie horen
                medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

                # loop over alle medewerkers om per medewerker gegevens te verzamelen
                for medewerker in medewerker_element:

                    foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                    foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)


                    # optioneel startdatum, telefoonnummer en mailadres
                    foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                    foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                    foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)

                    # genereer beschikbaarheidsgegevens

                    if organisatie_type != 'Provincies':
                        if foi_party != '':
                            bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                        else:
                            bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"

                    # vul de dictionary in
                    Dict = {
                            'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                            'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                            'dc_type': Type,
                            'dc_description': bereikbaarheidsgegevens,
                            'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{organisatie_type.lower()}.xml",
                            'dc_publisher': dc_publisher,
                            'dc_creator': "R0m4ndu",
                            'foi_retrievedDate': foi_retrievedDate,
                            'dc_date_year': dc_date_year,
                            'dc_publisher_name': dc_publisher_name,
#                             'foi_worksFor': f"nl.{dc_publisher}",
                            'foi_title': prefixes,
                            'foaf_initials': foaf_initials,
                            'foaf_firstName': foaf_firstName,
                            'foaf_lastName': foaf_lastName,
                            'foaf_name': name_without_prefix,
                            'foaf_mbox': foaf_mbox,
                            'foaf_phone': foaf_phone,
                            'foi_startDate': foi_startDate,
                            'foi_party': organisatie_naam,
                            'foi_function': foaf_function_type,
                            'foi_files': [] ,
                        }

                    # Verwijder alle lege strings uit de dict
                    filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                    # en voeg de dictionary toe aan 
                    DICT[foi_count] = filtered_dict

                    foi_count+=1
                
                if dc_publisher != '':
                
                    # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                    final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                                'infobox': {'foi_totalDossiers': len(DICT),
                                             'foi_dossiers': DICT}}


                    list_of_dicts.append(final_dict)

    number_of_people = sum(i['infobox']['foi_totalDossiers'] for i in list_of_dicts)
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts

In [12]:
people_num, dict_list = json_create_multi_layer('Hoog College van Staat')

organisatie = 'Hoog College van Staat'

tree = ET.parse(f"exportOO_full.xml")
namespaces = {
    'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)
list_of_peoples = [organisatie_element.findall('.//p:medewerker', namespaces) for organisatie_element in organisatie_elements]
people = sum(len(lst) for lst in list_of_peoples)

print(f'aantal medewerkers in XML: {people}')
print(f'aantal medewerkers extracted in JSON: {people_num}')


identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

for dl in dict_list:
    dossiers = dl['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1

print(id_dict)

C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\2265821717.py:26: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Hoog College van Staat"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


aantal medewerkers in XML: 317
aantal medewerkers extracted in JSON: 301
{'foi_function': 301, 'dc_publisher': 301, 'foaf_initials': 301, 'foaf_firstName': 4, 'foaf_lastName': 301, 'foi_title': 91, 'foaf_mbox': 6, 'foaf_phone': 0, 'foaf_type': 0, 'foi_startDate': 125, 'foi_party': 301}


C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\198372450.py:10: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Hoog College van Staat"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie}"]', namespaces)


In [13]:
D = {}

for i in dict_list:
    x = i['infobox']['foi_dossiers'][0]['dc_publisher_name']
    y = i['infobox']['foi_totalDossiers']
    if x not in D:
        D[x] = y
    else:
        D[x] += y
    
print(D)
print(sum(D.values()))
print(len(D))

{'Eerste Kamer der Staten-Generaal': 120, 'Kanselarij der Nederlandse Orden': 1, 'Tweede Kamer der Staten-Generaal': 180}
301
3


In [14]:
# def search_yahoo(Query):

#     time.sleep(1)
    
#     url = "http://search.yahoo.com/search?p=%s"
#     r = requests.get(url % Query) 
#     soup = BeautifulSoup(r.text)
#     hrefs = [a['href'] for a in soup.find_all('a', href=True)]
#     hrefs = [href for href in hrefs if 'yahoo' not in hrefs]

#     new_hrefs = [href for href in hrefs if "yahoo" not in href and '#' not in href]
    
#     return new_hrefs

In [16]:
# def extract_urls(hrefs, website_url, last_name):
    
#     URLs = {'twitter_url': '', 'account_url': '', 'linkedin_url': ''}
    
#     last_name = last_name.lower()
    
#     twitter = False
#     website = False
#     linkedin = False
    
#     for url in hrefs:
        
#         url = url.lower()
#         if 'twitter' in url and last_name in url and twitter == False:
#             twitter = True
#             URLs['twitter_url'] = url
            
#         if website_url in url and last_name in url and website == False:
#             website = True
#             URLs['account_url'] = url

#         if 'linkedin' in url and last_name in url and linkedin == False:
#             linkedin = True
#             URLs['linkedin_url'] = url
    
#     return URLs['twitter_url'], URLs['account_url'], URLs['linkedin_url']
        
# TESTER = extract_urls(new_hrefs, 'www.aaenmaas.nl', 'sennema')
# print(TESTER)

In [17]:
%%timeit

json_create_provincies('Provincies')

NameError: name 'json_create_provincies' is not defined

In [18]:
def create_functie_table(organisatie):
    if organisatie in ['Provincie', 'Ministerie']:
        people_num, dict_list = json_create_multi_layer(organisatie)
        
    else:    
        people_num, dict_list = json_create_single_layer(organisatie)

    data = []

    for d in dict_list:

        doi_split = d['resource'].split('.')
        publisher = d['infobox']['foi_dossiers'][0]['dc_publisher_name']
        functie = doi_split[2]
        personen = d['infobox']['foi_totalDossiers']


        functie_omzetten = {
            'Dijkgraaf': 'Dijk/Watergraaf',
            'Watergraaf': 'Dijk/Watergraaf',
            'Secretaris - Algemeen directeur': 'Secretaris-directeur',
            'Voorzitter': 'Dijk/Watergraaf',
            'Burgemeester waarnemend': 'Burgemeester',
            'Adjunct-hoofd': 'Adjunct hoofd',
            'Ádjunct-hoofdeenheid': 'Adjunct hoofd',
            'Commissaris van de koning': 'Commissaris van de Koning',
            'CdK': 'Commissaris van de Koning',
            'CvdK': 'Commissaris van de Koning'
            
        }

        if functie in functie_omzetten.keys():
            functie = functie_omzetten[functie]

        data.append({'Publisher': publisher, 'Functie': functie, 'Count': personen})

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Pivot the table
    table = pd.pivot_table(df, values='Count', columns='Functie', index='Publisher', aggfunc='sum', fill_value=0)

    return table


In [20]:
provincie_table = create_functie_table('Ministerie')

pd.set_option('display.max_columns', None)
display(provincie_table)
provincie_table.describe()
pd.reset_option('display.max_columns')

table_cols = provincie_table.columns.tolist()
print(len(table_cols))

p_sums = provincie_table.sum(axis=1)
display(p_sums)

C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\2265821717.py:26: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Ministerie"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


Functie,Adelingshoofd Tevens Plv,Adjunct-directeur,Administratief Medewerker Kabinetszaken,Adviseur / Plv Afdelingshoofd,Adviseur Bedrijfsvoering,Afd,Afdelingshoofd,Afdelingshoofd / Directiecontroller,Afdelingshoofd /Tevens Plv,Afdelingshoofd Advies/ Plv,Afdelingshoofd Afdeling Advies,Afdelingshoofd Arbeidszaken,Afdelingshoofd Benc/Plv,Afdelingshoofd Bestuur En Burger,Afdelingshoofd Fhid,Afdelingshoofd Ict/Ih,Afdelingshoofd Kienu,Afdelingshoofd OenI,Afdelingshoofd Tevens Plv,Afdelingshoofd/tevens plv,Algemeen Directeur,Algemeen commandant a,Algemeen directeur,Algemeen secretaris/directeur,Ambtelijk Secretaris Minister,Auditor / Teamleider,Beijing,Beveiligingsambtenaar,Beveiligingsambtenaar (BVA),Brigadecommandant Drentse-IJsselstreek,Brigadecommandant Brabant-Zuid,Brigadecommandant Caribisch gebied,Brigadecommandant District Brabant-Noord en Limburg,Brigadecommandant Grensbewaking Schiphol,Brigadecommandant Limburg-Zuid,Brigadecommandant Noord-Holland,Brigadecommandant Oostgrens-Midden,Brigadecommandant Scheldestromen,Brigadecommandant Veluwe,Brigadecommandant Waddengebied,Brigadecommandant Zuid-Holland,Brussel,CIORijk/directeur,Clusterhoofd,Clusterhoofd Ketenorganisatie,Commandant,Commandant 11 Luchtmobiele Brigade,Commandant 13 Lichte Brigade,Commandant 43 Gemechaniseerde Brigade,Commandant Air Operations Control Nw Milligen,Commandant Centrum voor Mens & Luchtvaart,Commandant Defensie Bedrijf Grondgebonden Luchtverdedigings Commando,Commandant Defensie Cyber Commando,Commandant Defensie Helikopter Commando & vlb,Commandant Explosieven Opruimingsdienst Defensie,Commandant Groep Luchtmacht Reserve,Commandant Joint Meteorologische Groep,Commandant Koninklijke Militaire School,Commandant Korps Commando Troepen,Commandant Landstrijdkrachten (CLAS),Commandant Logistiek Centrum Woensdrecht,Commandant Luchtstrijdkrachten (C-LSK),Commandant Materieellogistiek Commando,Commandant Operationeel Ondersteunings Commando Landmacht,"Commandant Opleidings, Training en Kenniscentrum",Commandant Opleidings- en Trainingscommando,Commandant Special Operations Command,Commandant der Strijdkrachten,Commandant der Zeestrijdkrachten (C-ZSK),Commandant vlb,Coordinator Bedrijfsvoering,Coordinerend Directeur Inkoop,Coördinator Kabinetszaken/ Chef Protocol,Coördinator Toezicht 2020,Coördinator/Fa Servicemanager,Coördinerend/Special,Dijk/Watergraaf,Directeur,Directeur,Directeur Bedrijfsvoering Politie/plv,Directeur Begroting,"Directeur Beschermen, Aanpakken en Voorkomen (DBAenV)",Directeur Besturing en Innovatie,Directeur Bestuursondersteuning (DBO),Directeur Contraterrorisme en Beveiliging Burgerluchtvaart,"Directeur Control, Bedrijfsvoering en Juridische Zaken (CBJ)",Directeur Directie Aansturen Operationele Gereedheid,Directeur Directie Managment Informatie en Beheer,Directeur Directie Sanctietoepassing en Jeugd,Directeur Financien en Control,Directeur Internationale Militaire Samenwerking,Directeur Kaderstelling en Inrichting,Directeur Kennis,Directeur Kennis en Advies,Directeur Monitoring en Beheer,Directeur NCTV met portefeuille Nationale Crisisbeheersing,Directeur Nationale Crisisbeheersing en Bewaken en Beveiligen,Directeur Operaties,Directeur Personeelsbeleid en Management,Directeur Plannen,Directeur Politieel beleid en taakuitvoering,Directeur Regie en Control,Directeur Rjz,Directeur Staf,"Directeur Strategie, Analyse en Bedrijfsvoering",Directeur Toezicht,Directeur Veiligheid,Directeur Werkgeverszaken,"Directeur, Lid Landelijke Directie",Directeur- Generaal Migratie,Directeur-Generaal,Directeur-Generaal Straffen en Beschermen,Directeur-Generaal (waarnemend),Directeur-Generaal Beleid,Directeur-Generaal Curatieve Zorg,Directeur-Generaal Langdurige Zorg,Directeur-Generaal Politie,Directeur-Generaal Politie en Veiligheidsregio's,Directeur-Generaal Rechtspleging en Rechtshandhaving,Directeur-Generaal Volksgezondheid,Directeur-generaal,Directiesecretariaat,Directiesecretaris,Functionaris Gegevensbescherming (FG),Geneve,Hoofd,"Hoofd , plv",Hoofd / P

267


Publisher
Binnenlandse Zaken en Koninkrijksrelaties     55
Defensie                                     103
Economische Zaken en Klimaat                   4
Financiën                                      2
Justitie en Veiligheid                       255
Landbouw, Natuur en Voedselkwaliteit           4
Onderwijs, Cultuur en Wetenschap             165
Sociale Zaken en Werkgelegenheid               1
Volksgezondheid, Welzijn en Sport             99
dtype: int64

In [21]:
waterschap_table = create_functie_table('Waterschap')
display(waterschap_table)
YYY = waterschap_table.describe()
rows_to_drop = ['25%', '50%', '75%']
YYY = YYY.drop(rows_to_drop)

YYY.to_latex()

C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\2287353622.py:28: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Waterschap"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


Functie,Dijk/Watergraaf,Secretaris-directeur
Publisher,,
Aa en Maas,1,1
"Amstel, Gooi en Vecht",1,1
Brabantse Delta,1,1
De Dommel,1,1
Drents Overijsselse Delta,1,1
Hollandse Delta,1,1
Hoogheemraadschap De Stichtse Rijnlanden,1,1
Hoogheemraadschap Hollands Noorderkwartier,1,1
Hoogheemraadschap van Delfland,1,1


'\\begin{tabular}{lrr}\n\\toprule\nFunctie &  Dijk/Watergraaf &  Secretaris-directeur \\\\\n\\midrule\ncount &             21.0 &                  21.0 \\\\\nmean  &              1.0 &                   1.0 \\\\\nstd   &              0.0 &                   0.0 \\\\\nmin   &              1.0 &                   1.0 \\\\\nmax   &              1.0 &                   1.0 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [22]:
import numpy as np

gemeente_table = create_functie_table('Gemeente')
pd.set_option('display.max_rows', None)
display(gemeente_table)
print(len(gemeente_table))

print(gemeente_table['Raadslid'].sum())

pd.reset_option('display.max_rows')
XXX = gemeente_table.describe()
rows_to_drop = ['25%', '50%', '75%']
XXX = XXX.drop(rows_to_drop)

XXX.to_latex()

C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\2287353622.py:28: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/p:organisaties/p:organisatie/[p:types = "Gemeente"]'
  organisatie_elements = tree.findall(f'//p:organisaties/p:organisatie/[p:types = "{organisatie_type}"]', namespaces)


Functie,Burgemeester,Fractievoorzitter,Gemeentesecretaris,Locoburgemeester,Raadsgriffier,Raadslid,Wethouder
Publisher,,,,,,,
Gemeente 's-Hertogenbosch,1,16,1,1,1,39,6
Gemeente Aa en Hunze,1,6,1,1,1,21,3
Gemeente Aalsmeer,1,6,1,0,1,23,2
Gemeente Aalten,1,7,1,1,1,21,4
Gemeente Achtkarspelen,1,8,1,1,1,21,4
Gemeente Alblasserdam,1,6,1,1,1,19,4
Gemeente Albrandswaard,1,7,1,1,1,22,3
Gemeente Alkmaar,1,13,1,1,1,39,6
Gemeente Almelo,1,15,1,1,1,35,4


344
8543


'\\begin{tabular}{lrrrrrrr}\n\\toprule\nFunctie &  Burgemeester &  Fractievoorzitter &  Gemeentesecretaris &  Locoburgemeester &  Raadsgriffier &    Raadslid &   Wethouder \\\\\n\\midrule\ncount &    344.000000 &         344.000000 &          344.000000 &        344.000000 &     344.000000 &  344.000000 &  344.000000 \\\\\nmean  &      1.002907 &           8.154070 &            0.976744 &          0.956395 &       1.002907 &   24.834302 &    4.119186 \\\\\nstd   &      0.053916 &           2.617966 &            0.150934 &          0.204511 &       0.053916 &    8.081862 &    1.195367 \\\\\nmin   &      1.000000 &           3.000000 &            0.000000 &          0.000000 &       1.000000 &    9.000000 &    1.000000 \\\\\nmax   &      2.000000 &          16.000000 &            1.000000 &          1.000000 &       2.000000 &   45.000000 &    9.000000 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [23]:
from collections import Counter

def json_alle_organisaties():
    org_dict = {}
    
    Types = []
    TOOIs = []
    
    count = 0
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',
                   'Provincie': 'pv',
                   'Ministerie': 'mnre',
                   'Regionaal samenwerkingsorgaan': 'so',
                  }
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }
    
    Type = 'Gemeente'
    
    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall(f'/p:organisaties/p:organisatie', namespaces)
#     organisatie_elements = tree.findall(f'./p:organisaties/p:organisatie/[p:types = "{Type}"]', namespaces)
    
    for organisatie  in organisatie_elements:
        
        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        Types.append(Type)
        
        TOOI = organisatie.find('.//p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces)
        Code = organisatie.find('.//p:identificatiecodes/p:resourceIdentifier[@p:naam="Organisatiecode"]', namespaces)
            
        if TOOI is not None:
            TOOI = TOOI.text.split('/')[-1]
            TOOI_org = re.sub(r'\d+', '', TOOI)
            TOOIs.append(TOOI_org)
            
        else:
            if Code is not None:
                Code = Code.text
                Code_org = re.sub(r'\d+', '', Code)
                if Code_org != '':
                    TOOIs.append(Code_org)
        
            
        
        if Type in afkortingen:
            afkorting = afkortingen[Type]
        else:
            afkorting = ''
    
        Dict = create_organisatie_json(organisatie, namespaces, afkorting)
        org_dict[count] = Dict
        
        count+=1
        
    final_dict = {'infobox':
                    {
                        'foi_totalDossiers': count,
                        'foi_dossiers': org_dict,
                    }
                 }
        
    
    Json = json.dumps(final_dict, indent=4)

    # Write JSON string to a text file
    with open(f"alle_organisaties_json.txt", "w") as file:
        file.write(Json)
    
    return final_dict, Types, TOOIs
    
FD, Types, TOOIs = json_alle_organisaties()


'Meest voorkomende organisatie types'
for i in enumerate(Counter(Types).most_common(len(Counter(Types)))):
    print(f"{i[0]+1}. {i[1][0]}: {i[1][1]}")
    
print('')
    
'Meest voorkomende organisatie TOOIs'
for i in enumerate(Counter(TOOIs).most_common(len(Counter(TOOIs)))):
    print(f"{i[0]+1}. {i[1][0]}: {i[1][1]}")
    
print('')
    
identifiers = ['dc_publisher', 'foaf_mbox', 'foaf_phone', 'foi_website', 'foi_fax', 'foi_startDate', 'foi_endDate', 'foi_bezoekadres', 
               'foi_postadres', 'foi_lastUpdate', 'foaf_type']

identifiers = list(set(Types))
id_dict = {key: [] for key in identifiers}

# dossiers = FD['infobox']['foi_dossiers']
# for k,v in dossiers.items():
#     for ID in identifiers:
#         if ID in v:
#             id_dict[ID] += 1
        
                
# print(id_dict)

for k,v in dossiers.items():
    ud = dossiers[k]['foi_lastUpdate']
    
    date = datetime.strptime(ud, '%Y-%m-%d')

    current_date = datetime.now()
    time_difference = current_date - date
    days_ago = time_difference.days
    
    Type = dossiers[k]['foaf_type']
    
    if Type in ['Gemeente', 'Waterschap', 'Ministerie', 'Provincie']:
        if 'dc_publisher' in dossiers[k]:
            id_dict[Type].append(days_ago)
    else:
        id_dict[Type].append(days_ago)
        
for k,v in id_dict.items():
    if v != []:
        id_dict[k] = sum(v) / len(v)
        
print(id_dict)
    

C:\Users\ramon\AppData\Local\Temp\ipykernel_29596\1332371288.py:29: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './p:organisaties/p:organisatie'
  organisatie_elements = tree.findall(f'/p:organisaties/p:organisatie', namespaces)


1. Organisatie met overheidsbemoeienis: 1376
2. Regionaal samenwerkingsorgaan: 421
3. Gemeente: 419
4. Zelfstandig bestuursorgaan: 140
5. Adviescollege: 111
6. Agentschap: 30
7. Rechtspraak: 25
8. Waterschap: 25
9. Ministerie: 12
10. Provincie: 12
11. Grensoverschrijdend regionaal samenwerkingsorgaan: 11
12. Interdepartementale commissie: 7
13. Openbaar lichaam voor beroep en bedrijf: 6
14. Hoog College van Staat: 6
15. Koepelorganisatie: 4
16. Organisatieonderdeel: 3
17. Caribisch openbaar lichaam: 3
18. Politie en brandweer: 2
19. Kabinet van de Koning: 1

1. gm: 411
2. so: 407
3. oorg: 127
4. zb: 110
5. ws: 21
6. mnre: 12
7. pv: 12
8. col: 3



KeyError: 'foi_lastUpdate'

In [ ]:
sorted_dict = dict(sorted(id_dict.items(), key=lambda item: item[1], reverse=False))

sorted_dict

# # Extract labels and values from the sorted dictionary
# labels = sorted_dict.keys()
# values = sorted_dict.values()

# # Plotting the data
# plt.barh(range(len(sorted_dict)), values)
# plt.yticks(range(len(sorted_dict)), labels)
# plt.xlabel('Duration')
# plt.ylabel('Organization Type')
# plt.title('Dagen sinds laatste update')
# plt.tight_layout()
# plt.show()

In [ ]:
df = pd.DataFrame.from_dict(id_dict, orient='index', columns=['Count'])
# df['Gem'] = id_dict.values()


display(df)

In [ ]:
import matplotlib.pyplot as plt

keys = id_dict.keys()
values = id_dict.values()

plt.bar(keys, values)
plt.xlabel('Keys')
plt.ylabel('Values')
plt.title('Dictionary Graph')
plt.xticks(rotation=90)
plt.show()

In [ ]:
def json_medewerkers(Type = ''):
    
    DICT = {}
    
    foi_count = 0
    
    tree = ET.parse("exportOO_full.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }
    
    afkortingen = {'Waterschap': 'ws',
                  'Gemeente': 'gm',
                   'Provincie': 'pv',
                   'Ministerie': 'mnre'
                  }
    

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]
    
    Type = 'Gemeente'

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    if Type != '':
        organisatie_elements = tree.findall(f'./p:organisaties/p:organisatie/[p:types = "{Type}"]', namespaces)
    
    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        if Type in afkortingen:
            afkorting = afkortingen[Type]
        else:
            afkorting = ''

        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 

        
        # in het functie_element staan alle verschillende functies
        functie_element = organisatie.findall('p:functies/p:functie', namespaces)

        # loop over deze functies
        for functie in functie_element:

            foaf_function_type = xpath(functie, 'p:naam', namespaces)

            # vindt alle medewerkers die bij deze functie horen
            medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

            # loop over alle medewerkers om per medewerker gegevens te verzamelen
            for medewerker in medewerker_element:
                
                foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)
                
                # optioneel startdatum, telefoonnummer en mailadres
                foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)
                
                foi_bezoekadres = xpath_adres(medewerker, namespaces, 'Bezoek')
                
                foi_postadres = xpath_adres(medewerker, namespaces, 'Post')
            
                # vul de dictionary in
                Dict = {
                        'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                        'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                        'foi_function': foaf_function_type,
                        'dc_source': f"https://organisaties.overheid.nl/archive/exportOO_{Type.lower()}.xml",
                        'dc_creator': "Ramon Duursma",
                        'foi_retrievedDate': foi_retrievedDate,
                        'dc_date_year': dc_date_year,
                        'dc_publisher': dc_publisher,
                        'dc_publisher_name': dc_publisher_name,
                        'foaf_initials': foaf_initials,
                        'foaf_firstName': foaf_firstName,
                        'foaf_lastName': foaf_lastName,
                        'foi_title': prefixes,
                        'foaf_name': name_without_prefix,
                        'foaf_mbox': foaf_mbox,
                        'foaf_phone': foaf_phone,
                        'foi_bezoekadres': foi_bezoekadres,
                        'foi_postadres': foi_postadres,
#                         'foi_twitter': foi_twitter,
#                         'foi_website': foi_website,
#                         'foi_linkedin': foi_linkedin,
                        'foaf_type': Type,
                        'foi_startDate': foi_startDate,
                        'foi_party': foi_party,
                        'foi_files': [] ,
                    }

                # Verwijder alle lege strings uit de dict
                filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                # en voeg de dictionary toe aan 
                DICT[foi_count] = filtered_dict

                foi_count+=1
            
    # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
    final_dict = {
                'infobox': {'foi_totalDossiers': len(DICT),
                             'foi_dossiers': DICT}}
                
            

    number_of_people = final_dict['infobox']['foi_totalDossiers']
    
    Json = json.dumps(final_dict, indent=4)

    # Write JSON string to a text file
    with open(f"alle_medewerkers_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, final_dict


NP, fd = json_medewerkers()

In [ ]:

identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foi_bezoekadres', 'foi_postadres' , 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

dossiers = fd['infobox']['foi_dossiers']
for k,v in dossiers.items():
    for ID in identifiers:
        if ID in v:
            id_dict[ID] += 1
            
print(id_dict)

In [ ]:
df = pd.DataFrame.from_dict(id_dict, orient='index', columns=['Totaal'])
df['Zonder Gem'] = df['Totaal'] - df['Gem']
display(df)


In [ ]:
all_types = counts['thing'].to_list()
all_counts = []

tree = ET.parse("exportOO_full.xml")

# Define the namespaces used in the XML document
namespaces = {
'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

for Type in all_types:
    
    mc = 0


    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    org_elements = tree.findall(f'./p:organisaties/p:organisatie/[p:types = "{Type}"]', namespaces)
    
    for org in org_elements:
        medewerkers = org.findall('.//p:medewerkers/p:medewerker', namespaces)
        
        mc += len(medewerkers)
    
    
    all_counts.append(mc)
    
medewerker_counts = dict(zip(all_types, all_counts))
print(medewerker_counts)



In [ ]:
thing_series = pd.Series(Types)

# Use value_counts() to get the count of each thing
counts = thing_series.value_counts().reset_index()

# Rename the columns
counts.columns = ['Type Organisatie', 'aantal']

display(counts)

In [ ]:
counts['medewerkers'] = medewerker_counts.values()

new_row = pd.DataFrame({'Type Organisatie': ['Totaal:'], 'aantal': [sum(counts['aantal'])], 'medewerkers': [sum(counts['medewerkers'])]})

counts = counts.append(new_row, ignore_index=False)

counts.index = counts.index+1

counts['m/o'] = counts['medewerkers'] / counts['aantal']

counts['m/o'] = counts['m/o'].round(1)

display(counts)